In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.tree            import DecisionTreeRegressor
from sklearn.neural_network  import MLPRegressor
from sklearn.linear_model    import LinearRegression
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics         import mean_squared_error
from sklearn.metrics         import r2_score

In [13]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [144]:
train_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\Train.csv")

In [208]:
test_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\Test.csv")

In [146]:
train_df.isnull().sum()

id                                   0
name                                12
host_id                              0
host_name                           13
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       6982
reviews_per_month                 6982
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [147]:
train_df.select_dtypes(exclude=['object'])

id    host_id  latitude  longitude  price  minimum_nights  \
0       5728806    4271676  40.72217  -73.99481    120               7   
1       2243769   11460768  40.80020  -73.96045   1500               1   
2      35515415  267193767  40.75558  -73.89316    200               2   
3      36202006   43392243  40.60110  -74.07830     30               2   
4       3780951   16065171  40.81022  -73.94266    115               1   
...         ...        ...       ...        ...    ...             ...   
34221  28342248  120730056  40.70602  -73.90485     80               5   
34222   8438775   41042927  40.77697  -73.97698    137               1   
34223  27332770   80381355  40.86912  -73.92183    195               2   
34224   9063972    3040551  40.69709  -73.91435     80               3   
34225  13372860   16755566  40.69076  -73.92886     69               4   

       number_of_reviews  reviews_per_month  calculated_host_listings_count  \
0                      3               0.06                               3   
1                      0                NaN                               1   
2                      4               4.00                               1   
3                      0                NaN                               4   
4                     16               0.27                               1   
...                  ...                ...                             ...   
34221                  2               0.21                               1   
34222                163               3.55                               1   
34223                  6               0.53                               1   
34224                  0                NaN                               1   
34225                  5               0.13                               1   

       availability_365  
0                     0  
1                     0  
2                   365  
3                    82  
4                   365  
...                 ...  
34221                 0  
34222                32  
34223                 0  
34224                 0  
34225                 0  

[34226 rows x 10 columns]

In [148]:
train_df.columns 

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [149]:
test_df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [150]:
len(train_df.neighbourhood.unique()) , len(train_df.neighbourhood_group.unique()), len(test_df.neighbourhood.unique()) , len(test_df.neighbourhood_group.unique())

(217, 5, 207, 5)

In [151]:
def removal_of_outliers(df,room_t, nhood, distance):
    '''Function removes outliers that are above 3rd quartile and below 1st quartile'''
    '''The exact cutoff distance above and below can be adjusted'''

    new_piece = df[(df["room_type"]==room_t)&(df["neighbourhood_group"]==nhood)]["log_price"]
    #defining quartiles and interquartile range
    q1 = new_piece.quantile(0.25)
    q3 = new_piece.quantile(0.75)
    IQR=q3-q1

    trimmed = df[(df.room_type==room_t)&(df["neighbourhood_group"]==nhood) &(df.log_price>(q1-distance*IQR))&(df.log_price<(q3+distance*IQR))]
    return trimmed

In [152]:
def date_replacement(date):
    if date <=3:
        return "Last_review_last_three_day"
    elif date <= 7:
        return "Last_review_last_week"
    elif date <= 30:
        return "Last_review_last_month"
    elif date <= 183:
        return "Last_review_last_half_year"
    elif date <= 365:
        return "Last_review_last year"
    elif date <= 1825:
        return "Last_review_last_5_years"
    else:
        return "Last_review_never"

In [153]:
def PreProcessing(df):
    #fill missing values for last review and reviews per month with 0
    df[["last_review", "reviews_per_month"]] = df[["last_review", "reviews_per_month"]].fillna(0)
    
    #if there is no host name or listing name fill in None
    df[["name", "host_name"]] = df[["name", "host_name"]].fillna("None")
    df = df.drop(['latitude','longitude'],axis=1)
    
    #Drop rows were price of the listing is 0. We are not intersted in "free" 
    #listings as they are most likely an error.
    if 'price' in df.columns:
        free = len(df[df.price == 0])
        df = df[df.price != 0].copy()
    
    df = df[df["minimum_nights"] <=31].copy()
    #df = df.drop(['id','host_id'],axis=1)
    
    #separate out numerical variables
    a=pd.DataFrame(df.dtypes.copy())
    b= a[a[0] != 'object'].reset_index()
    #drop id and host id:
    numeric_vars= df.select_dtypes(exclude=['object'])
    
    #for num in numeric_vars:
    #    df["log_("+num+" +1)"] = np.log(df[num]+1)
        
    if 'price' in df.columns:
        df["log_price"] = np.log1p(df.price)
    df=df.drop(columns = numeric_vars[2:]).copy()
    
    #numeric_vars = df.columns.tolist()[6:8]+df.columns.tolist()[10:]
    #x=df[numeric_vars].apply(lambda x: np.log(np.abs(x+1))).corr(method='pearson')
    
    #separate out numerical variables
    #a=pd.DataFrame(df.dtypes.copy())
    #b= a[a[0] == 'object'].reset_index()
    #drop id and host id:
    #non_num=b["index"].tolist()
    
    grouped = df.groupby("neighbourhood")
    
    if 'price' in df.columns:
        price_grouped = grouped["log_price"]
        price = price_grouped.agg([np.mean,np.median,np.max, np.std]).sort_values("mean")
    
    #One hot encoding
    #df = pd.concat([df, pd.get_dummies(df["neighbourhood"], drop_first=False)], axis=1)
    #save neighborhoods into a list for further analysis:
    #neighborhoods = df.neighbourhood.values.tolist()
    boroughs = df.neighbourhood_group.unique().tolist()
    #drop the neighbourhood column from the database
    df.drop(['neighbourhood'],axis=1, inplace=True)
    
    grouped = df.groupby("room_type")
    if 'price' in df.columns:
        room_type_price_grouped = grouped["log_price"]
        room_type_price = room_type_price_grouped.agg([np.mean,np.median,np.max, np.std]).sort_values("mean")
        room_type_price
    
    if 'price' in df.columns:
        #apply the function
        df_private = pd.DataFrame()
        for neighborhood in boroughs:
            a = removal_of_outliers(df, "Private room",neighborhood,3)
            df_private = df_private.append(a)

        df_shared = pd.DataFrame()
        for neighborhood in boroughs:
            a = removal_of_outliers(df, "Shared room",neighborhood,3)
            df_shared = df_shared.append(a)
    
        df_apt = pd.DataFrame()
        for neighborhood in boroughs:
            a = removal_of_outliers(df, "Entire home/apt",neighborhood,3)
            df_apt = df_apt.append(a)
    
        # Create new dataframe to absorb newly produced data    
        df_old=df.copy()    
        df = pd.DataFrame()
        df = df.append([df_private,df_shared,df_apt])

    grouped = df.groupby("room_type")
    if 'price' in df.columns:
        room_type_price_grouped = grouped["log_price"]
        room_type_price = room_type_price_grouped.agg([np.mean,np.median,np.max, np.std]).sort_values("mean")
    
    #convert room types to dummies
    df = pd.concat([df, pd.get_dummies(df["room_type"], drop_first=False)], axis=1)
    df.drop(['room_type'],axis=1, inplace=True)
    
    #convert object to datetime:
    df["last_review"] = pd.to_datetime(df["last_review"])
    
    df["last_review"]=df["last_review"].apply(lambda x: dt.datetime(2019,7,8)-x)
    df["last_review"]=df["last_review"].dt.days.astype("int").replace(18085, 1900)
    
    df["last_review"]=df["last_review"].apply(lambda x: date_replacement(x))
    grouped = df.groupby("last_review")
    if 'price' in df.columns:
        last_review_price_grouped = grouped["log_price"]
        last_review_price = last_review_price_grouped.agg([np.mean,np.median,np.max, np.std]).sort_values("mean")
    
    #convert last review to dummies
    df = pd.concat([df, pd.get_dummies(df["last_review"], drop_first=False)], axis=1)
    df.drop(["last_review"],axis=1, inplace=True)

     #convert last review to dummies
    df = pd.concat([df, pd.get_dummies(df['neighbourhood_group'], drop_first=False)], axis=1)
    df.drop(['neighbourhood_group'],axis=1, inplace=True)
    
    #drop unnecessary columns
    df = df.drop(['name','host_name'], axis=1).copy()
    #copy for later
    #df2 = df.copy()
    return df
    

In [154]:
train_df = PreProcessing(train_df)

In [156]:
train_df.columns

Index(['log_price', 'Entire home/apt', 'Private room', 'Shared room',
       'Last_review_last year', 'Last_review_last_5_years',
       'Last_review_last_half_year', 'Last_review_last_month',
       'Last_review_last_three_day', 'Last_review_last_week',
       'Last_review_never', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island'],
      dtype='object')

In [155]:
train_df = train_df.drop(['log_(id +1)','log_(host_id +1)'],axis=1)

KeyError: "['log_(id +1)' 'log_(host_id +1)'] not found in axis"

In [157]:
train_df.columns

Index(['log_price', 'Entire home/apt', 'Private room', 'Shared room',
       'Last_review_last year', 'Last_review_last_5_years',
       'Last_review_last_half_year', 'Last_review_last_month',
       'Last_review_last_three_day', 'Last_review_last_week',
       'Last_review_never', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island'],
      dtype='object')

target = train_df['log_price'].copy()
#drop unnecessary columns
train_df = train_df.drop(['log_price'], axis=1).copy()
#strip the target column from input columns and put it in front
train_df = pd.concat([target, train_df], axis=1).copy()
#select input variable columns
nums = train_df.iloc[:,1:]

In [158]:
train_df

log_price  Entire home/apt  Private room  Shared room  \
0       4.795791                0             1            0   
1       7.313887                1             0            0   
2       5.303305                1             0            0   
3       3.433987                0             0            1   
4       4.753590                0             0            1   
...          ...              ...           ...          ...   
34221   4.394449                1             0            0   
34222   4.927254                0             1            0   
34223   5.278115                1             0            0   
34224   4.394449                0             1            0   
34225   4.248495                1             0            0   

       Last_review_last year  Last_review_last_5_years  \
0                          0                         1   
1                          0                         0   
2                          0                         0   
3                          0                         0   
4                          0                         0   
...                      ...                       ...   
34221                      1                         0   
34222                      0                         0   
34223                      0                         0   
34224                      0                         0   
34225                      0                         1   

       Last_review_last_half_year  Last_review_last_month  \
0                               0                       0   
1                               0                       0   
2                               0                       0   
3                               0                       0   
4                               1                       0   
...                           ...                     ...   
34221                           0                       0   
34222                           0                       1   
34223                           1                       0   
34224                           0                       0   
34225                           0                       0   

       Last_review_last_three_day  Last_review_last_week  Last_review_never  \
0                               0                      0                  0   
1                               0                      0                  1   
2                               1                      0                  0   
3                               0                      0                  1   
4                               0                      0                  0   
...                           ...                    ...                ...   
34221                           0                      0                  0   
34222                           0                      0                  0   
34223                           0                      0                  0   
34224                           0                      0                  1   
34225                           0                      0                  0   

       Bronx  Brooklyn  Manhattan  Queens  Staten Island  
0          0         0          1       0              0  
1          0         0          1       0              0  
2          0         0          0       1              0  
3          0         0          0       0              1  
4          0         0          1       0              0  
...      ...       ...        ...     ...            ...  
34221      0         0          0       1              0  
34222      0         0          1       0              0  
34223      0         0          1       0              0  
34224      0         1          0       0              0  
34225      0         1          0       0              0  

[33824 rows x 16 columns]

In [183]:
features = train_df.drop(['log_price'],axis=1).columns

In [190]:
'log_price ~'+ '+'.join(features)

'log_price ~Entire home/apt+Private room+Shared room+Last_review_last year+Last_review_last_5_years+Last_review_last_half_year+Last_review_last_month+Last_review_last_three_day+Last_review_last_week+Last_review_never+Bronx+Brooklyn+Manhattan+Queens+Staten Island'

In [185]:

from patsy import dmatrices


In [195]:

# get y and X dataframes based on this regression:
u, C = dmatrices('log_price ~ Last_review_last_5_years+Last_review_last_half_year+Last_review_last_month+Last_review_last_three_day+Last_review_last_week+Last_review_never+Bronx+Brooklyn+Manhattan+Queens', train_df, return_type='dataframe')

In [196]:
VIF = pd.DataFrame()

In [197]:
VIF['Features']  = C.columns

In [198]:
VIF['VIF'] = [variance_inflation_factor(C.values , i) for i in range(C.shape[1])]

In [199]:
VIF

Features         VIF
0                    Intercept  145.978251
1     Last_review_last_5_years    2.379324
2   Last_review_last_half_year    2.230295
3       Last_review_last_month    2.555164
4   Last_review_last_three_day    1.361222
5        Last_review_last_week    1.476727
6            Last_review_never    2.420776
7                        Bronx    3.979501
8                     Brooklyn   33.807786
9                    Manhattan   34.451965
10                      Queens   15.033249

In [200]:
X=train_df.drop(['log_price','Bronx','Brooklyn','Manhattan','Queens'],axis=1)
y=train_df['log_price']

In [127]:
X=train_df.drop(['log_(price +1)'],axis=1)
y=train_df['log_(price +1)']

In [128]:
train_df.isnull().sum()


log_(price +1)                             0
log_(minimum_nights +1)                    0
log_(number_of_reviews +1)                 0
log_(reviews_per_month +1)                 0
log_(calculated_host_listings_count +1)    0
log_(availability_365 +1)                  0
Entire home/apt                            0
Private room                               0
Shared room                                0
Last_review_last year                      0
Last_review_last_5_years                   0
Last_review_last_half_year                 0
Last_review_last_month                     0
Last_review_last_three_day                 0
Last_review_last_week                      0
Last_review_never                          0
Bronx                                      0
Brooklyn                                   0
Manhattan                                  0
Queens                                     0
Staten Island                              0
dtype: int64

In [201]:
#y= target
#x = nums
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=24)

In [202]:
X.head()

Entire home/apt  Private room  Shared room  Last_review_last year  \
0                0             1            0                      0   
1                1             0            0                      0   
2                1             0            0                      0   
3                0             0            1                      0   
4                0             0            1                      0   

   Last_review_last_5_years  Last_review_last_half_year  \
0                         1                           0   
1                         0                           0   
2                         0                           0   
3                         0                           0   
4                         0                           1   

   Last_review_last_month  Last_review_last_three_day  Last_review_last_week  \
0                       0                           0                      0   
1                       0                           0                      0   
2                       0                           1                      0   
3                       0                           0                      0   
4                       0                           0                      0   

   Last_review_never  Staten Island  
0                  0              0  
1                  1              0  
2                  0              0  
3                  1              1  
4                  0              0

In [203]:
from sklearn.linear_model import LinearRegression

In [204]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_train = lr.predict(X_train)
print('Train RMSE:',np.sqrt(mean_squared_error(y_train, lr.predict(X_train))))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, lr.predict(X_test))))


Train RMSE: 0.5321551963439077
Test RMSE: 0.540581351769211


## LinearReg TEST PREDICTION

In [209]:
test_df.head()

id                                               name    host_id  \
0   1525602                    Perfect Temporary Brooklyn Home    1200603   
1  30430185                        BIG APPLE - COMFY KING ROOM  224414117   
2  21354525  Cozy Brooklyn Heights Getaway w/ Manhattan Access   11743513   
3  35995074  Amazing*Quiet*Sunny*Bedroom*PrivateLivingRm*Ch...    4128829   
4  34392081  Hello! This is a very cozy space in Williamsburg.  259630588   

  host_name neighbourhood_group     neighbourhood  latitude  longitude  \
0    Andrea            Brooklyn     Crown Heights  40.66751  -73.95867   
1   Gabriel           Manhattan    Hell's Kitchen  40.75655  -73.99690   
2     Henry            Brooklyn  Brooklyn Heights  40.69252  -73.99121   
3      Sara              Queens  Ditmars Steinway  40.77292  -73.90101   
4     Alina            Brooklyn      Williamsburg  40.71863  -73.94980   

         room_type  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt               2                121  2019-05-31   
1     Private room               1                 18  2019-05-05   
2     Private room               1                 87  2019-06-16   
3     Private room               9                  0         NaN   
4     Private room               1                 28  2019-06-26   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.70                               1                39  
1               2.49                              30               364  
2               4.29                               1               108  
3                NaN                               2               365  
4              14.00                               2                20

In [210]:
test_id_col = test_df['id']

In [211]:
test_df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [212]:
test_df = PreProcessing(test_df)

In [213]:
test_df.columns

Index(['Entire home/apt', 'Private room', 'Shared room',
       'Last_review_last year', 'Last_review_last_5_years',
       'Last_review_last_half_year', 'Last_review_last_month',
       'Last_review_last_three_day', 'Last_review_last_week',
       'Last_review_never', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island'],
      dtype='object')

In [138]:
test_df = test_df.drop(['log_(id +1)','log_(host_id +1)'],axis=1)

In [214]:
X.columns


Index(['Entire home/apt', 'Private room', 'Shared room',
       'Last_review_last year', 'Last_review_last_5_years',
       'Last_review_last_half_year', 'Last_review_last_month',
       'Last_review_last_three_day', 'Last_review_last_week',
       'Last_review_never', 'Staten Island'],
      dtype='object')

In [215]:
test_df.columns

Index(['Entire home/apt', 'Private room', 'Shared room',
       'Last_review_last year', 'Last_review_last_5_years',
       'Last_review_last_half_year', 'Last_review_last_month',
       'Last_review_last_three_day', 'Last_review_last_week',
       'Last_review_never', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island'],
      dtype='object')

In [218]:
test_df = test_df.drop(['Bronx','Brooklyn','Manhattan','Queens'],axis=1)

In [219]:
X.shape ,test_df.shape

((33824, 11), (14516, 11))

In [220]:
test_df['price'] = lr.predict(test_df)

In [221]:
test_df['price'] = np.exp(test_df['price'])-1

In [222]:
submissions_lr = pd.concat([test_id_col, test_df['price']], axis=1) 


In [223]:
submissions_lr.to_csv('submission_LinearReg.csv', index=False) 

In [224]:
submissions_lr

id       price
0       1525602  164.364859
1      30430185   70.540496
2      21354525   72.991859
3      35995074   83.977185
4      34392081   72.991859
...         ...         ...
14664  27730191  164.364859
14665  29676417   52.686109
14666   1813829   71.773619
14667  25017403  164.364859
14668  19057073  169.989399

[14669 rows x 2 columns]